In [46]:
import pyspark
import boto3

from io import StringIO 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

## CONFIG SESSÃO SPARK LOCAL

In [19]:
conf = pyspark.SparkConf()

# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
conf.setMaster("local[1]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", 'EVrYpCgbPdv0KWQV') \
    .set("spark.hadoop.fs.s3a.secret.key", 'Hlli13j51CZLRdGlFkO2RZzcIZ1hnOvt') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08-S33')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

In [23]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='EVrYpCgbPdv0KWQV',
    aws_secret_access_key='Hlli13j51CZLRdGlFkO2RZzcIZ1hnOvt',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [20]:
df_estados = spark.read.option("header", True).csv("municipios-estados/csv/estados.csv")
df_municipios = spark.read.option("header", True).csv("municipios-estados/csv/municipios.csv")

print(df_estados.show(), df_estados.printSchema())
print(df_municipios.show(), df_municipios.printSchema())

+---------+---+-------------------+--------+---------+--------+
|codigo_uf| uf|               nome|latitude|longitude|  regiao|
+---------+---+-------------------+--------+---------+--------+
|       11| RO|           Rondônia|  -10.83|   -63.34|   Norte|
|       12| AC|               Acre|   -8.77|   -70.55|   Norte|
|       13| AM|           Amazonas|   -3.47|    -65.1|   Norte|
|       14| RR|            Roraima|    1.99|   -61.33|   Norte|
|       15| PA|               Pará|   -3.79|   -52.48|   Norte|
|       16| AP|              Amapá|    1.41|   -51.77|   Norte|
|       17| TO|          Tocantins|   -9.46|   -48.26|   Norte|
|       21| MA|           Maranhão|   -5.42|   -45.44|Nordeste|
|       22| PI|              Piauí|    -6.6|   -42.28|Nordeste|
|       23| CE|              Ceará|    -5.2|   -39.53|Nordeste|
|       24| RN|Rio Grande do Norte|   -5.81|   -36.59|Nordeste|
|       25| PB|            Paraíba|   -7.28|   -36.72|Nordeste|
|       26| PE|         Pernambuco|   -8

In [21]:
df_estados.createOrReplaceTempView("ESTADOS")
df_municipios.createOrReplaceTempView("MUNICIPIOS")

In [22]:
df_join = spark.sql('''
SELECT 
  mun.*
, es.uf
FROM MUNICIPIOS AS mun
JOIN ESTADOS AS es ON mun.codigo_uf=es.codigo_uf
''')
print(df_join.show())

+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+---+
|codigo_ibge|               nome|latitude|longitude|capital|codigo_uf|siafi_id|ddd|     fuso_horario| uf|
+-----------+-------------------+--------+---------+-------+---------+--------+---+-----------------+---+
|    5200050|    Abadia de Goiás|-16.7573| -49.4412|      0|       52|    1050| 62|America/Sao_Paulo| GO|
|    3100104|Abadia dos Dourados|-18.4831| -47.3916|      0|       31|    4001| 34|America/Sao_Paulo| MG|
|    5200100|          Abadiânia| -16.197| -48.7057|      0|       52|    9201| 62|America/Sao_Paulo| GO|
|    3100203|             Abaeté|-19.1551| -45.4444|      0|       31|    4003| 37|America/Sao_Paulo| MG|
|    1500107|         Abaetetuba|-1.72183| -48.8788|      0|       15|    0401| 91|America/Sao_Paulo| PA|
|    2300101|            Abaiara|-7.34588| -39.0416|      0|       23|    1301| 88|America/Sao_Paulo| CE|
|    2900108|             Abaíra|-13.2488| -41

In [91]:
df_estados.write.csv("tarefa-aula-08/csv/estados.csv", header=True, mode="overwrite")
df_municipios.write.csv("tarefa-aula-08/csv/municipios.csv", header=True, mode="overwrite")

In [95]:
client.upload_file("tarefa-aula-08/csv/estados.csv/part-00000-e0be20d9-05fe-43e8-8b90-c3b9bf814dac-c000.csv", "tarefa-aula-08", "csv/estados.csv")
client.upload_file("tarefa-aula-08/csv/municipios.csv/part-00000-a327fc74-62fd-4dcb-9bca-5cec882edabe-c000.csv", "tarefa-aula-08", "csv/municipios.csv")

In [100]:
df_estados.write.json("tarefa-aula-08/json/estados.json", mode="overwrite")
df_municipios.write.json("tarefa-aula-08/json/municipios.json", mode="overwrite")

In [98]:
client.upload_file("tarefa-aula-08/json/estados.json/part-00000-69e3c5c1-f338-4f39-91d4-7b88ec692de4-c000.json", "tarefa-aula-08", "json/estados.json")
client.upload_file("tarefa-aula-08/json/municipios.json/part-00000-5b2d838c-7016-45e3-92cf-597e76f36d72-c000.json", "tarefa-aula-08", "json/municipios.json")

In [99]:
df_estados.write.parquet("tarefa-aula-08/parquet/estados.parquet", mode="overwrite")
df_municipios.write.parquet("tarefa-aula-08/parquet/municipios.parquet", mode="overwrite")

In [101]:
client.upload_file("tarefa-aula-08/parquet/estados.parquet/part-00000-8b3e4a5b-cdc5-4919-86fa-969a7246141b-c000.snappy.parquet", "tarefa-aula-08", "parquet/estados.parquet")
client.upload_file("tarefa-aula-08/parquet/municipios.parquet/part-00000-6580d635-a104-4446-ac2a-d6bf50818b52-c000.snappy.parquet", "tarefa-aula-08", "parquet/municipios.parquet")